# Inequalities testing

Этот код нужен, чтобы проверять придуманный сертификат на адекватность - введи ограничения на углы и задай вектор u в функции func(var) и код попробует найти углы, на которых знаки Au будут разные.

In [1]:
import sympy
import mystic.symbolic as ms
import numpy as np
import re
import operator
import pprint

In [2]:
var = list("ABCDEFG")
add_eqs = []

for _ in range(len(var)):
    s = ' + '.join(var[:4]) + ' - ' + ' - '.join(var[4:])
    add_eqs.append(s + '<= 3.141592653589793')
    add_eqs.append(s + '>= -3.141592653589793')
    var.append(var.pop(0))

add_eqs.append('A + B + C + D + E + F + G > 3.141592653589793')
add_eqs.append('A + B + C + D + E + F + G < 9.42477796076938')

add_eqs = '\n'.join(add_eqs)

In [3]:
def check_linear_constraints(var: dict, equations: str, verbose=False):
    
    comps = ['<', '>', '<=', '>=']
    COMPARES = {'<': operator.lt, '>': operator.gt, '<=': operator.le, '>=': operator.ge}
    OPERATIONS = {'+': operator.add, '-': operator.sub}
    RES = True
    
    for eq in equations.split('\n'):
        
        if not len(eq):
            continue
                
        # Choose a comparison that has to be done
        COMP = ''
        for comp in comps:
            if comp in eq:
                COMP = comp
        
        # Calculate the value on the left
        left, right = re.split('<=|>=|<|>',eq); left.strip(); right.strip()
        right_val = float(right)
        
        val = 0; op = '+'
        
        for el in left.split(' '):
            
            # Add an element
            if el in var.keys():
                #print(el, var[el])
                val = OPERATIONS[op](val, var[el])
            # Change operation
            elif el in OPERATIONS.keys():
                #print(op)
                op = el
            #print(val)
        # Make a comparison
        if verbose:
            if not COMPARES[COMP](val, right_val):
                print(eq)
                print(val, right_val)
                print(COMPARES[COMP](val, right_val))
        RES = RES and COMPARES[COMP](val, right_val)
    if verbose:
        print("Final: ", RES)
    
    return RES

In [4]:
equations = """
A + B < 3.141592653589793
A + B > 0
B + C < 3.141592653589793
B + C > 0
C + D < 3.141592653589793
C + D > 0
D + E < 3.141592653589793
D + E > 0
E + F < 3.141592653589793
E + F > 0
F + G < 3.141592653589793
F + G > 0
A + B + C + D - E - F - G < 0
A + B + C + D - E - F - G >= -3.141592653589793
E + F + G + A - B - C - D <= 3.141592653589793
E + F + G + A - B - C - D >= 0
A + B + C + D + E + F + G <= 9.42477796076938
A + B + C + D + E + F + G >= 6.283185307179586
B + C + D + E + F + G - A < 9.42477796076938
B + C + D + E + F + G - A > 6.283185307179586
"""
equations += add_eqs

In [5]:
eqns = ms.simplify(equations, variables=var, join="and_") + '\n'+ """A > 0
B > 0
C > 0
D > 0
E > 0
F > 0
G > 0"""

In [6]:
constrain = ms.generate_constraint(ms.generate_solvers(eqns, var))

In [7]:
def func(var: dict):
    
    A = np.array([
        [1, -np.cos(var['A']), -np.cos(var['E'] + var['F'] + var['G'])],
        [-np.cos(var['A']), 1, -np.cos(var['B'] + var['C'] + var['D'])],
        [-np.cos(var['E'] + var['F'] + var['G']), -np.cos(var['B'] + var['C'] + var['D']), 1]
    ])
    
    u = np.array([1,1,1])
    
    return A@u

In [ ]:
MAX_ITER = 1000

for _ in range(MAX_ITER):
    a = np.random.uniform(0, np.pi, 7)
    solution = constrain(a)
    sol = {k: v for k,v in zip(var, solution)}
    if check_linear_constraints(sol, equations, verbose=False):
        s = sum(func(sol) >= 0) 
        # Check that all signs are different
        if not (s == 0 or s == len(func(sol))):
            pprint.pprint(sol)
            print(func(sol))